In [ ]:
import os
import numpy as np
import random
import cv2
from tensorflow.keras.utils import Sequence
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress bars
from sklearn.model_selection import train_test_split

# --- Load Dataset ---


In [ ]:
# Define your dataset paths (adjust the paths as needed)
dataset_dirs = [
    '/kaggle/input/youtube-faces-with-facial-keypoints/youtube_faces_with_keypoints_full_1/youtube_faces_with_keypoints_full_1',
    '/kaggle/input/youtube-faces-with-facial-keypoints/youtube_faces_with_keypoints_full_2/youtube_faces_with_keypoints_full_2',
    '/kaggle/input/youtube-faces-with-facial-keypoints/youtube_faces_with_keypoints_full_3/youtube_faces_with_keypoints_full_3',
    '/kaggle/input/youtube-faces-with-facial-keypoints/youtube_faces_with_keypoints_full_4/youtube_faces_with_keypoints_full_4'
]

# Collecting all files from the specified directories
files = []
for dataset_dir in dataset_dirs:
    files += [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# Debugging step: print the number of files found
print(f"Number of files found: {len(files)}")
if len(files) == 0:
    print("No files found! Please check the directory paths and file structure.")
else:
    # Perform train-test split if files are found
    train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

    print(f"Training set size: {len(train_files)} files")
    print(f"Validation set size: {len(val_files)} files")

# --- Data Generator Class ---


In [ ]:
class DataGenerator(Sequence):
    def __init__(self, files, batch_size=32, sample_ratio=0.1, img_size=(256, 256), shuffle=True, return_names=False):
        self.files = files
        self.batch_size = batch_size
        self.sample_ratio = sample_ratio
        self.img_size = img_size
        self.shuffle = shuffle
        self.return_names = return_names
        
        if self.shuffle:
            random.shuffle(self.files)

    def __len__(self):
        return int(np.floor(len(self.files) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.files[index * self.batch_size:(index + 1) * self.batch_size]

        images, bboxes, landmarks_2d, landmarks_3d, image_names = [], [], [], [], []
        for npz_file in batch_files:
            data = np.load(npz_file)
            color_images = data['colorImages']
            bboxes_data = data['boundingBox']
            landmarks2D_data = data['landmarks2D']
            landmarks3D_data = data['landmarks3D']

            num_frames = color_images.shape[-1]
            sampled_indices = random.sample(range(num_frames), int(self.sample_ratio * num_frames))

            filename = os.path.basename(npz_file).split('.')[0]

            for idx in sampled_indices:
                img = color_images[..., idx]
                img = cv2.resize(img, self.img_size)
                img = img / 255.0  # Normalize

                images.append(img)  # Use original images

                bboxes.append(bboxes_data[..., idx])
                landmarks_2d.append(landmarks2D_data[..., idx])
                landmarks_3d.append(landmarks3D_data[..., idx])
                image_names.append(filename)

        images = np.array(images)

        if self.return_names:
            return images, images, image_names  # Return image names for visualization
        else:
            return images, images  # Don't return names during training

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.files)

# --- Data Generators ---


In [ ]:
train_generator = DataGenerator(files=train_files, batch_size=8, sample_ratio=0.05, img_size=(256, 256), shuffle=True)
val_generator = DataGenerator(files=val_files, batch_size=8, sample_ratio=0.05, img_size=(256, 256), shuffle=False)

# --- SR-GAN Model Components ---


In [ ]:
def build_generator():
    input_img = layers.Input(shape=(256, 256, 3))

    # Initial Convolutional Layer
    x = layers.Conv2D(64, (9, 9), padding='same')(input_img)
    x = layers.Activation('relu')(x)

    # Residual Blocks
    for _ in range(16):
        res = layers.Conv2D(64, (3, 3), padding='same')(x)
        res = layers.BatchNormalization()(res)
        res = layers.Activation('relu')(res)
        res = layers.Conv2D(64, (3, 3), padding='same')(res)
        res = layers.BatchNormalization()(res)
        x = layers.add([x, res])

    # Upsampling Layers
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # Final Convolutional Layer
    x = layers.Conv2D(3, (9, 9), padding='same')(x)
    output_img = layers.Activation('tanh')(x)

    generator = keras.Model(inputs=input_img, outputs=output_img, name="Generator")
    return generator

def build_discriminator():
    input_img = layers.Input(shape=(1024, 1024, 3))

    x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same')(input_img)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(512, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1024)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    validity = layers.Dense(1, activation='sigmoid')(x)

    discriminator = keras.Model(inputs=input_img, outputs=validity, name="Discriminator")
    return discriminator

# --- Build SR-GAN ---


In [ ]:
generator = build_generator()
discriminator = build_discriminator()

In [ ]:
# Compile Models
discriminator.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')
discriminator.trainable = False  # Freeze the discriminator during generator training

In [ ]:
# Combined Model
input_img = layers.Input(shape=(256, 256, 3))
generated_img = generator(input_img)
validity = discriminator(generated_img)

combined_model = keras.Model(inputs=input_img, outputs=[generated_img, validity])
combined_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5), loss=['mean_squared_error', 'binary_crossentropy'])

# --- Training Loop with Callbacks ---


In [ ]:
# Callbacks
callbacks = [
    ModelCheckpoint('/kaggle/working/best_srgan_model.keras', save_best_only=True, monitor='val_loss', mode='min'),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
]

# Training Loop
epochs = 50
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch in tqdm(train_generator):
        # Generate high-resolution images from low-resolution images
        low_res_images, _ = batch
        high_res_images = low_res_images  # Use the same images for training the generator

        # Train the Discriminator
        valid = np.ones((len(low_res_images), 1))  # Real labels
        fake = np.zeros((len(low_res_images), 1))  # Fake labels

        d_loss_real = discriminator.train_on_batch(high_res_images, valid)
        d_loss_fake = discriminator.train_on_batch(generator.predict(low_res_images), fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the Generator
        g_loss = combined_model.train_on_batch(low_res_images, [high_res_images, valid])

    print(f"[Epoch {epoch + 1}/{epochs}] [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss[0]} | G validity loss: {g_loss[1]}]")

# --- Visualize Results ---


In [ ]:
def visualize_results(generator, low_res_images, image_names, n=5):
    plt.figure(figsize=(20, 10))
    for i in range(n):
        plt.subplot(2, n, i + 1)
        plt.imshow(low_res_images[i])
        plt.title(f"Low-Resolution Image\n{image_names[i]}")
        plt.axis("off")

        high_res_image = generator.predict(low_res_images[i:i + 1])
        plt.subplot(2, n, i + 1 + n)
        plt.imshow(high_res_image[0])
        plt.title(f"Generated High-Resolution Image\n{image_names[i]}")
        plt.axis("off")
    plt.show()

# Sample results from the training
sample_batch = train_generator[0]  # Get the first batch
sample_low_res_images, sample_image_names = sample_batch  # Get both images and names
visualize_results(generator, sample_low_res_images, sample_image_names)

In [ ]:
# Save the generator model
generator.save('/kaggle/working/srgan_generator_model.keras')